In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
path = '/content/drive/MyDrive/MANU 465 Team 7/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [ ]:
# Importing the dataset and flatten the images
folder = 'Imageset_Combined'
images = []
EI = []
strengths = [57.70520833,
20.240625,
32.43,
22.63020833,
64.28,
17.95,
17.67,
39.44,
9.33,
40.68,
26.56,
32.69,
57.93,
67.88,
222.46]

total_path = path + folder
for filename in os.listdir(total_path):
    # read grayscale image
    img = cv2.imread(os.path.join(total_path,filename),0)
    print(img.shape)
    # Get beam number
    beam = int(filename.split('_')[1][1:])
    print(beam)
    if img is not None:
        images.append(img.flatten())
        EI.append(strengths[beam-1])


images = np.array(images)
EI = np.array(EI)
print(len(images))
print(images.shape)

(3901, 512)
1
(3901, 512)
1
(3901, 512)
1
(3901, 512)
1
(3901, 512)
1
(3901, 512)
1
(3901, 512)
1
(3901, 512)
2
(3901, 512)
2
(3901, 512)
2
(3901, 512)
2
(3901, 512)
1
(3901, 512)
2
(3901, 512)
2
(3901, 512)
2
(3901, 512)
3
(3901, 512)
3
(3901, 512)
3
(3901, 512)
3
(3901, 512)
2
(3901, 512)
4
(3901, 512)
3
(3901, 512)
4
(3901, 512)
3
(3901, 512)
3
(3901, 512)
4
(3901, 512)
3
(3901, 512)
4
(3901, 512)
4
(3901, 512)
4
(3901, 512)
5
(3901, 512)
4
(3901, 512)
4
(3901, 512)
5
(3901, 512)
5
(3901, 512)
5
(3901, 512)
5
(3901, 512)
5
(3901, 512)
5
(3901, 512)
5
(3901, 512)
6
(3901, 512)
6
(3901, 512)
6
(3901, 512)
6
(3901, 512)
6
(3901, 512)
6
(3901, 512)
6
(3901, 512)
7
(3901, 512)
7
(3901, 512)
7
(3901, 512)
7
(3901, 512)
6
(3901, 512)
7
(3901, 512)
8
(3901, 512)
8
(3901, 512)
8
(3901, 512)
7
(3901, 512)
7
(3901, 512)
7
(3901, 512)
8
(3901, 512)
9
(3901, 512)
8
(3901, 512)
8
(3901, 512)
8
(3901, 512)
8
(3901, 512)
9
(3901, 512)
9
(3901, 512)
9
(3901, 512)
9
(3901, 512)
9
(3901, 512)
10
(3901

In [ ]:
# Implement PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
pca.fit(images)
print(pca.explained_variance_ratio_)
print(pca.singular_values_)
# Get the principal components
PC = pca.transform(images)
print(PC.shape)


[0.11820943 0.01871667 0.01752947 0.01740006 0.01703357 0.01689859
 0.01686081 0.01569635 0.01532393 0.01521766 0.01504283 0.01483427
 0.01429749 0.01411625 0.01288341 0.0126726  0.01238585 0.01214449
 0.01138192 0.01099667]
[628555.5707355  250110.49321844 242048.31521685 241153.15645945
 238600.04667016 237652.73550917 237386.92367996 229042.97284715
 226309.48718807 225523.36985958 224224.15455366 222664.34464092
 218598.70640013 217208.70098723 207507.10640403 205802.46249681
 203460.72990389 201468.55855361 195040.73683244 191711.5163189 ]
(120, 20)


In [ ]:
# Set 20% of the data aside for testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(PC, EI, test_size = 0.2, random_state = 0)

In [ ]:
# Set up ANN model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.utils import np_utils

In [ ]:
# Set up CNN model for images
print(images.shape[1])
model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))
model.add(Flatten())
model.add(Dense(units = 20, activation = 'relu'))
model.add(Dense(units = 40, activation = 'relu'))
model.add(Dense(units = 20, activation = 'relu'))
model.add(Dense(units = 10, activation = 'relu'))
model.add(Dense(units = 1, activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

1997312
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 20)                0         
                                                                 
 dense_2 (Dense)             (None, 20)                420       
                                                                 
 dense_3 (Dense)             (None, 40)                840       
                                                                 
 dense_4 (Dense)             (None, 20)                820       
                                                                 
 dense_5 (Dense)             (None, 10)                210       
                                                                 
 dense_6 (Dense)             (None, 1)                 11        
                                                                 
Total params: 2,301
Trainable params: 2,301
No

In [ ]:
# Fit CNN to the images
model.fit(X_train, y_train, batch_size = 10, epochs = 25)

Epoch 1/25
10/10 [==============================] - 1s 4ms/step - loss: -409445.7500 - accuracy: 0.0000e+00
Epoch 2/25
10/10 [==============================] - 0s 4ms/step - loss: -809386.5000 - accuracy: 0.0000e+00
Epoch 3/25
10/10 [==============================] - 0s 4ms/step - loss: -1489621.3750 - accuracy: 0.0000e+00
Epoch 4/25
10/10 [==============================] - 0s 4ms/step - loss: -2259661.7500 - accuracy: 0.0000e+00
Epoch 5/25
10/10 [==============================] - 0s 4ms/step - loss: -3600970.7500 - accuracy: 0.0000e+00
Epoch 6/25
10/10 [==============================] - 0s 4ms/step - loss: -5332712.5000 - accuracy: 0.0000e+00
Epoch 7/25
10/10 [==============================] - 0s 5ms/step - loss: -8004587.5000 - accuracy: 0.0000e+00
Epoch 8/25
10/10 [==============================] - 0s 4ms/step - loss: -11881927.0000 - accuracy: 0.0000e+00
Epoch 9/25
10/10 [==============================] - 0s 4ms/step - loss: -17263754.0000 - accuracy: 0.0000e+00
Epoch 10/25
10/10 [

In [ ]:
# Save the model
model.save('modelMANU.h5')

In [ ]:
# Load the model
from keras.models import load_model
model = load_model('modelMANU.h5')

In [ ]:
# Predict the EI    
y_pred = model.predict(X_test)

In [ ]:
# Evaluate the model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score

print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))
print('R2 Score:', r2_score(y_test, y_pred))
print('Explained Variance Score:', explained_variance_score(y_test, y_pred))